## SUPPLY CHAIN NETWORK DESIGN TO SUPPORT BIOFUEL PRODUCTION
<p> A company has decided to produce bioethanol in the state of Texas. The company needs to design a supply chain consisting of suppliers, hubs and biorefineries for the conversion of raw material (i.e., biomass) into biofuel. </p>
<p> The potential locations to open hubs correspond to train stations because the transportation mode utilized to move the raw material from the hubs to the biorefineries is train, while truck is the transportation mode utilized to move the biomass from the counties to the hubs.</p>
<p> This project is to minimize the investment and transportation costs by finding the optimal number of hubs and biorefineries that the company needs to install as well as the flows between suppliers-hubs and hubs-biorefineries.</p>


<p> PuLP is the Python library installed, imported and used in this project for linear optimization </p>

In [1]:
# Import the necessary libraries
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import pulp
from pulp import*

## Data Importation and Cleaning

In [2]:
suppliers = pd.read_csv('TX_suppliers.csv')
suppliers['supply'] = round(suppliers.supply, 2)
suppliers = suppliers.drop(['index'], axis =1)
supplies = suppliers.supply
total_supply = sum(suppliers.supply)

hubs = pd.read_csv('TX_hubs.csv')
hubs = hubs.drop(['index'], axis =1)
Hub_Capacity = hubs.capacity
Hub_Invest = hubs.invest.iloc[0]
Hubs = hubs['hub']

road = pd.read_csv('TX_roads.csv')
road['cost'] = round(road.cost, 2)
road = road.drop(['index'], axis =1)

plants = pd.read_csv('TX_plants.csv')
Yield = plants['yield'].iloc[0]
plants['plt_capacity'] = plants.capacity/plants['yield']
plants = plants.drop(['index'], axis =1)
Plt_Invest = plants.invest.iloc[0]

rail = pd.read_csv('TX_railroads.csv')
rail['cost'] = round(rail.cost, 2)
rail['plt_invest'] = len(rail)*[plants.invest.iloc[0]]
rail = rail.drop(['index'], axis =1)
loading = rail.loading

network = pd.read_csv('TX_network.csv')
Demand = network.demand.iloc[0]
demand = Demand/Yield

unmet_Demand = demand - total_supply

## Introducing 3rd Party Supplier
<p> Since the sum of the county supplies can not meet the demand, a 3rd party supplier is introduced. </p>

In [3]:
suppliers.loc[254] = ['3_party', unmet_Demand]
suppliers['supply'] = round(suppliers.supply, 2)
suppliers

,county,supply
0,48001,13131.97
1,48003,1177.35
2,48005,3854.62
3,48007,308.18
4,48009,19802.14
5,48011,5603.76
6,48013,13962.20
7,48015,17336.34
8,48017,10562.02
9,48019,12501.36


In [4]:
# Adding 3rd Party Supplier to Hubs list
n = 1303
County = ['3_party'] * n
Distance = [0] * n
Cost = [0] * n
Third = {'county': County, 'distance':Distance, 'cost':Cost}
Third = pd.DataFrame(Third)
Third['hubs'] = Hubs
Third

,cost,county,distance,hubs
0,0,3_party,0,131
1,0,3_party,0,199
2,0,3_party,0,229
3,0,3_party,0,512
4,0,3_party,0,560
5,0,3_party,0,635
6,0,3_party,0,636
7,0,3_party,0,637
8,0,3_party,0,638
9,0,3_party,0,639


In [5]:
road = road.append(Third, ignore_index = True) 
road

,cost,county,distance,hubs
0,197.28,48001,1218.9736,131
1,53.03,48001,299.0238,199
2,147.18,48001,899.4465,229
3,53.21,48001,300.1810,512
4,53.04,48001,299.0745,560
5,192.24,48001,1186.8121,635
6,192.22,48001,1186.6888,636
7,178.89,48001,1101.6754,637
8,115.41,48001,696.8238,638
9,118.62,48001,717.3399,639


In [6]:
# Setting the indices
hubs = hubs.set_index(['hub'])
suppliers = suppliers.set_index(['county'])
plants = plants.set_index(['plant'])
road = road.set_index(['county', 'hubs'])
rail = rail.set_index(['hubs','plant'])

## Data Exploration and Optimization

In [8]:
# Decision Variables Defintion
road_supply = pulp.LpVariable.dicts('road_supply', [(i, j) for i in suppliers.index
                                                    for j in hubs.index], lowBound=0, cat='Continuous')

hub_status = pulp.LpVariable.dicts("hub_status", [j for j in hubs.index], cat='Binary')

rail_supply = pulp.LpVariable.dicts("rail_supply",[(j, k) for j in hubs.index
                                                   for k in plants.index], lowBound=0, cat='Continuous')

plt_status = pulp.LpVariable.dicts("plt_status",[k for k in plants.index], cat='Binary')

## Model 1

In [9]:
# Model 1 Initialization
model_1 = pulp.LpProblem("cost minimising supply-hub network", pulp.LpMinimize)

In [10]:
# Objective Function 1
model_1 += pulp.lpSum(
    [[road_supply[i, j] * road.loc[(i, j), 'cost']]  + 
     [hub_status[j] * hubs.loc[j, 'invest']] for i in suppliers.index for j in hubs.index])

In [11]:
# Demand Constraint
model_1 += pulp.lpSum([road_supply[i, j] for i in suppliers.index for j in hubs.index]) == demand

In [12]:
# Hub Capacity Constraint
for j in hubs.index:
    model_1 += pulp.lpSum([road_supply[i, j] for i in suppliers.index]) <= hubs.loc[j, 'capacity'] * hub_status[j]

In [14]:
# Model 1 Status
model_1.solve()
pulp.LpStatus[model_1.status]

'Optimal'

In [37]:
# Model 1 Result
cost_1 = pulp.value(model_1.objective)

amount_1 = "${:,.2f}".format(cost_1)

print('The optimal supply-hub cost is ' + amount_1)

The optimal hub-plant cost is $19,501,588,590.00


In [16]:
# Model 1 Table Formulation
rd_output = []
for i, j in road_supply:
    var_output = {
        'county': i,
        'hubs': j,
        'road_supply': road_supply[(i, j)].varValue,
        'hub_status': hub_status[j].varValue
    }
    rd_output.append(var_output)
rd_output_df = pd.DataFrame.from_records(rd_output).sort_values(['county', 'hubs'])
rd_output_df.set_index(['county', 'hubs'], inplace=True)
rd_output_df

hub_status  road_supply
county  hubs                          
48001   131           1.0          0.0
        199           0.0          0.0
        229           0.0          0.0
        512           0.0          0.0
        560           0.0          0.0
        635           0.0          0.0
        636           0.0          0.0
        637           0.0          0.0
        638           0.0          0.0
        639           0.0          0.0
        640           0.0          0.0
        17164         1.0          0.0
        17165         1.0          0.0
        17166         1.0          0.0
        17167         1.0          0.0
        17168         1.0          0.0
        17169         1.0          0.0
        17170         1.0          0.0
        17171         1.0          0.0
        17172         1.0          0.0
        17173         1.0          0.0
        17174         1.0          0.0
        17175         1.0          0.0
        17176         1.0          0.0
        17177         1.0          0.0
        17178         1.0          0.0
        17179         1.0          0.0
        17180         0.0          0.0
        17181         1.0          0.0
        17182         1.0          0.0
...                   ...          ...
3_party 18457         0.0          0.0
        18458         0.0          0.0
        18459         0.0          0.0
        18460         0.0          0.0
        18461         0.0          0.0
        18462         0.0          0.0
        18463         0.0          0.0
        18464         0.0          0.0
        18465         0.0          0.0
        18466         0.0          0.0
        18467         0.0          0.0
        18468         0.0          0.0
        18469         0.0          0.0
        18470         0.0          0.0
        18471         0.0          0.0
        18472         0.0          0.0
        18473         0.0          0.0
        18474         0.0          0.0
        18475         0.0          0.0
        18476         0.0          0.0
        18477         0.0          0.0
        18478         0.0          0.0
        18479         0.0          0.0
        18480         0.0          0.0
        18481         0.0          0.0
        18482         0.0          0.0
        18483         0.0          0.0
        18484         0.0          0.0
        18485         0.0          0.0
        18486         0.0          0.0

[332265 rows x 2 columns]

## Model 1 Output

In [17]:
rd = []
for i, j in road_supply:
    if rd_output_df.hub_status[i, j] == 1 and rd_output_df.road_supply[i, j] > 0:
        output = {
            'hubs': j,
            'road_supply': rd_output_df.road_supply[i, j],
            'hub_status': rd_output_df.hub_status[i, j] 
        }
        rd.append(output)
rd = pd.DataFrame.from_records(rd).sort_values('hubs')
rd.set_index('hubs', inplace=True)
rd

,hub_status,road_supply
hubs,,
131,1.0,300000.000
17164,1.0,300000.000
17165,1.0,300000.000
17166,1.0,300000.000
17167,1.0,300000.000
17168,1.0,300000.000
17169,1.0,300000.000
17170,1.0,300000.000
17171,1.0,300000.000


In [29]:
# Model 1 Document Exportation
rd.to_excel('Optimal Hubs.xlsx')

## Model 2

In [18]:
# Model 2 Initialization
model_2 = pulp.LpProblem("cost minimising hub-plant network", pulp.LpMinimize)

In [19]:
# Model 2 Objective Function
model_2 += pulp.lpSum(
    [[rail_supply[j, k] * rail.loc[(j, k), 'cost']] + 
     [plt_status[k] * plants.loc[k, 'invest']] for j in rd.index for k in plants.index])

In [20]:
# Model 2 Demand Constraint
model_2 += pulp.lpSum([rail_supply[j, k] for j in rd.index for k in plants.index]) == demand

In [21]:
# Plant Capacity Constraint
for k in plants.index:
    model_2 += pulp.lpSum([rail_supply[j, k] for j in rd.index]) <= plants.loc[k, 'plt_capacity'] * plt_status[k]

In [22]:
# Model 2 Status
model_2.solve()
pulp.LpStatus[model_2.status]

'Optimal'

In [38]:
# Model 2 Result
cost_2 = pulp.value(model_2.objective)

amount_2 = "${:,.2f}".format(cost_2)

print('The optimal hub-plant cost is ' + amount_2)

The optimal hub-plant cost is $28,810,495,340.00


In [24]:
# Model 2 Table Formulation
rl_output = []
for j, k in rail_supply: 
    var_output = {
        'hubs': j,
        'plant': k,
        'rail_supply': rail_supply[(j, k)].varValue,
        'plt_status': plt_status[k].varValue
    }
    rl_output.append(var_output)
rl_output_df = pd.DataFrame.from_records(rl_output).sort_values(['hubs', 'plant'])
rl_output_df.set_index(['hubs', 'plant'], inplace=True)
rl_output_df

plt_status  rail_supply
hubs  plant                         
131   541           0.0          0.0
      542           0.0          0.0
      543           0.0          0.0
      544           0.0          0.0
      545           0.0          0.0
      546           0.0          0.0
      9039          0.0          0.0
      9040          0.0          0.0
      9041          0.0          0.0
      9043          0.0          0.0
      9044          0.0          0.0
      9045          0.0          0.0
      9046          0.0          0.0
      9047          0.0          0.0
      9048          0.0          0.0
      9049          0.0          0.0
      9050          0.0          0.0
      9051          0.0          0.0
      9053          0.0          0.0
      9054          0.0          0.0
      9055          0.0          0.0
      9056          0.0          0.0
      9057          0.0          0.0
      9059          0.0          0.0
      9060          0.0          0.0
      9061          0.0          0.0
      9062          0.0          0.0
      9063          0.0          0.0
      9064          0.0          0.0
      9065          0.0          0.0
...                 ...          ...
18486 9183          0.0          NaN
      9184          0.0          NaN
      9185          1.0          NaN
      9186          0.0          NaN
      9187          0.0          NaN
      9188          0.0          NaN
      9189          0.0          NaN
      9190          0.0          NaN
      9192          0.0          NaN
      9193          0.0          NaN
      9194          0.0          NaN
      9195          0.0          NaN
      9197          0.0          NaN
      9198          0.0          NaN
      9199          0.0          NaN
      9200          0.0          NaN
      9202          0.0          NaN
      9203          0.0          NaN
      9204          0.0          NaN
      10056         0.0          NaN
      10057         0.0          NaN
      10058         1.0          NaN
      10059         0.0          NaN
      10060         0.0          NaN
      10061         0.0          NaN
      10062         0.0          NaN
      10063         0.0          NaN
      10065         0.0          NaN
      10066         0.0          NaN
      10067         0.0          NaN

[217601 rows x 2 columns]

## Model 2 Ouput

In [25]:
rl = []
for j, k in rail_supply:
    if rl_output_df.plt_status[j, k] == 1 and rl_output_df.rail_supply[j, k] > 0:
        output = {
            'plants': k,
            'rail_supply': rl_output_df.rail_supply[j, k],
            'plt_status': rl_output_df.plt_status[j, k] 
        }
        rl.append(output)
rl = pd.DataFrame.from_records(rl).sort_values('plants')
rl.set_index('plants', inplace=True)
rl

,plt_status,rail_supply
plants,,
9083,1.0,655447.00
9100,1.0,655447.00
9113,1.0,464384.73
9114,1.0,655447.00
9115,1.0,655447.00
9117,1.0,655447.00
9119,1.0,655447.00
9136,1.0,655447.00
9185,1.0,655447.00


In [30]:
# Model 2 Document Exportation
rl.to_excel('Optimal Plants.xlsx')

## Result

In [39]:
# Total Optimal Cost
total_cost = cost_1 + cost_2

total_amount = "${:,.2f}".format(total_cost)

print('The total optimal network cost is ' + total_amount)

The total optimal network cost is $48,312,083,930.00
